In [1]:
autosave 60

Autosaving every 60 seconds


In [2]:
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
from pandas_datareader._utils import RemoteDataError
from pandas_datareader import data as pdrr
import fix_yahoo_finance as yf
import imp
#import MySQLdb as msql
import pymysql as pymysql
import pymysql.cursors

In [3]:
con = pymysql.connect('localhost', 'root', 'Capstone', 'portfolio_management')
ticker=[]

with con:
    
    cur = con.cursor()
    cur.execute("SELECT * FROM twentyfivecompanies")

    rows = cur.fetchall()

    for row in rows:
        ticker.append(row[1])

In [4]:
yf.pdr_override() 

#ticker = ['AAPL','MSFT','GOOGL','GOOG','FB','V','MA','CSCO','INTC','ORCL','NVDA','IBM','ADBE','CRM','ACN','QCOM','TXN','PYPL','AVGO','ADP','ATVI','INTU','MU','CTSH','HPQ']
start = dt.datetime(2018,10,19)
end = dt.datetime(2018,11,30)

stock_data = pdr.get_data_yahoo(ticker, start=start, end=end)

In [5]:
for a in ['Adj Close', 'Volume','High','Low','Open']:
    del stock_data[a]

In [6]:
stock_export = stock_data['Close']

In [7]:
stock_export.head(1)

Symbols,AAPL,ADP,ATVI,CSCO,FB,GOOG,GOOGL,HP,IBM,INTC,MA,MSFT,NVDA,ORCL,PYPL,V
Date,,,,,,,,,,,,,,,,
2018-10-19,219.309998,143.410004,69.75,45.34,154.050003,1096.459961,1105.180054,66.150002,129.100006,44.0,203.059998,108.660004,229.169998,47.59,84.779999,140.080002


In [8]:
stock_export = stock_export.reset_index()

### Collecting Index Data

In [4]:
yf.pdr_override() 

ticker_idx = ['^DJI','^GSPC']
start = dt.datetime(2018,10,19)
end = dt.datetime(2018,11,30)

index_data = pdr.get_data_yahoo(ticker_idx, start=start, end=end)

In [5]:
for a in ['Adj Close', 'Volume','High','Low','Open']:
    del index_data[a]

In [6]:
index_export = index_data['Close']

In [7]:
index_export.to_csv('index_export.csv')

In [8]:
index_export = index_export.reset_index()

### Connecting to MySQL DB

#### Loading Benchmark Data - snp500 and dowjones

In [14]:
Date=[]
DJI=[]
GSPC=[]

for i in range(0,len(index_export)):
    Date.append(index_export.iloc[i]['Date'].strftime("%Y-%m-%d"))
    DJI.append(float(index_export.iloc[i]['^DJI']))
    GSPC.append(float(index_export.iloc[i]['^GSPC']))

In [15]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Capstone',
                             db='portfolio_management',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Create a new record
        sql = "INSERT INTO indexdata (value_date, dowjones, snp500) VALUES (%s,%s,%s)"
        for i in range(0,len(index_export)):
            cursor.execute(sql,(Date[i],DJI[i],GSPC[i]))

    # connection is not autocommit by default. So you must commit to save
    # your changes.
    connection.commit()

finally:
    connection.close()

In [16]:
sys_date = dt.datetime.now()

#### Loading Stock data (Company)

In [9]:
colname = list(stock_export.columns.values)

In [10]:
Date2=[]
COL2=[]
TICKR_SYMBOL=[]

for a in range(1,len(colname)):
    for i in range(0,len(stock_export)):
        Date2.append(stock_export.iloc[i]['Date'].strftime("%Y-%m-%d"))
        COL2.append(float(stock_export.iloc[i][colname[a]]))
        TICKR_SYMBOL.append(colname[a])
    #ACN.append(float(stock_export.iloc[i]['ACN']))

In [11]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='Capstone',
                             db='portfolio_management',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Create a new record
        sql = "INSERT INTO stockdata (value_date, stockprice, tickersymbol) VALUES (%s,%s,%s)"
        for i in range(0,len(COL2)):
            cursor.execute(sql,(Date2[i],COL2[i],TICKR_SYMBOL[i]))

    # connection is not autocommit by default. So you must commit to save
    # your changes.
    connection.commit()

finally:
    connection.close()